### 8.2 Customizing string formatting

In [1]:
_formats = {
    'ymd': '{d.year}-{d.month}-{d.day}',
    'mdy': '{d.month}/{d.day}/{d.year}',
    'dmy': '{d.day}/{d.month}/{d.year}'
}

In [2]:
class Date:

    def __init__(self, day, month, year):
        self.day , self.month, self.year = day, month, year
        
    def __format__(self, code):
        if code=='':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)

In [3]:
d = Date(31, 7, 1985)
print(format(d, 'dmy'))
print(format(d, 'mdy'))
print('{:ymd}'.format(d))

31/7/1985
7/31/1985
1985-7-31


### 8.8 Extending a Property in a Subclass

In [4]:
class Person:
    
    def __init__(self, name):
        self.name = name
        
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string.')
        self._name = value

In [5]:
class SubPerson(Person):
    
    @property
    def name(self):
        print('Getting name')
        return super().name
    
    
    @name.setter
    def name(self, value):
        print('Setting name')
        # https://stackoverflow.com/questions/38661438/python-super-two-argument-version-in-context-of-new
        super(SubPerson, SubPerson).name.__set__(self, value) # Having a class as the second argument is to access the __set__ class method

In [6]:
s = Person('Guido')
print(s.name)
s.name = 'BDFL'
print(s.name)

Guido
BDFL


In [7]:
l = SubPerson('Larry')
print(l.name)
l.name = 'Larry Wall'
print(l.name)

Setting name
Getting name
Larry
Setting name
Getting name
Larry Wall


In [8]:
class AnotherPerson(Person):
    @Person.name.getter
    def name(self):
        print('Getting name for AnotherPerson')
        return super().name
        

In [9]:
c = AnotherPerson('Bjarne')
print(c.name)
c.name += ' Stroustrop' # Will call the getter again
print(c.name)

Getting name for AnotherPerson
Bjarne
Getting name for AnotherPerson
Getting name for AnotherPerson
Bjarne Stroustrop


In [10]:
class A:
    def __init__(self):
        print('A: {}'.format(self))
        
class B(A):
    def __init__(self):
        print('Init B')
        super().__init__()
        print('B: {}'.format(self))
        
    def fake_init(self):
        print('B fake init')
        
b = B()

Init B
A: <__main__.B object at 0x111239d30>
B: <__main__.B object at 0x111239d30>


### Descriptors

In [11]:
class Typed:
    def __init__(self, name, expected_type):
        self.name = name
        self.expected_type = expected_type
        
    def __get__(self, instance, cls):
        if instance is None:
            return self
        return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError('{} requires an argument of type {!s}'.format(self.name, self.expected_type))
        instance.__dict__[self.name] = value

In [12]:
def typeassert(**kwargs):
    def decorate(cls):
        for name, expected_type in kwargs.items():
            setattr(cls, name, Typed(name, expected_type))
        return cls
    return decorate

In [13]:
@typeassert(name=str, shares=int, price=float)
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

In [14]:
vars(Stock)

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Stock.__init__(self, name, shares, price)>,
              '__dict__': <attribute '__dict__' of 'Stock' objects>,
              '__weakref__': <attribute '__weakref__' of 'Stock' objects>,
              '__doc__': None,
              'name': <__main__.Typed at 0x111240710>,
              'shares': <__main__.Typed at 0x111240748>,
              'price': <__main__.Typed at 0x111240780>})

In [15]:
google = Stock('GOOG', 2, 1234.)
try:
    amazon = Stock('AMZN', 3, '1989')
except TypeError as err:
    print('Error: {}'.format(err))

Error: price requires an argument of type <class 'float'>


### 8.10 Lazy Properties

In [16]:
# A lazy property implemented as a descriptor
class lazyproperty:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
            setattr(instance, self.func.__name__, value)
            return value

In [17]:
import math

class Circle:
    def __init__(self, radius):
        self.radius = radius
        
    @lazyproperty
    def area(self):
        print('Computing area')
        return math.pi*(self.radius**2)

In [18]:
c = Circle(4.0)
print(vars(c))
print()
print(c.area)
print()
print(vars(c))
print()
c.area = 42
print('Area is now mutable: {}'.format(c.area))

{'radius': 4.0}

Computing area
50.26548245743669

{'radius': 4.0, 'area': 50.26548245743669}

Area is now mutable: 42


In [19]:
def immutable_lazyproperty(func):
    name = '_lazy_' + func.__name__
    
    @property
    def lazy(self):
        if hasattr(self, name):
            return getattr(self, name)
        else:
            value = func(self)
            setattr(self, name, value)
            return value
    return lazy

In [20]:
class Square:
    def __init__(self, side):
        self.side = side
        
    @immutable_lazyproperty
    def area(self):
        print('Computing area')
        return self.side**2

In [21]:
c = Square(4.0)
print(vars(c))
print()
print(c.area)
print()
print(vars(c))
print()
try:
    c.area = 42
except Exception as err:
    print('Can not arbitrary change area now')


{'side': 4.0}

Computing area
16.0

{'side': 4.0, '_lazy_area': 16.0}

Can not arbitrary change area now
